In [1]:
import torch
import torch.nn.functional as F

from transformers import AutoModel, AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("facebook/xglm-1.7B")
# model = AutoModel.from_pretrained("facebook/xglm-1.7B")


/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
num = 256008 + 56
for i in range(1, 65):
    if(i+num) % 64 == 0:
        print(i)

64


In [4]:
model = AutoModel.from_pretrained("facebook/xglm-1.7B")

Some weights of the model checkpoint at facebook/xglm-1.7B were not used when initializing XGLMModel: ['lm_head.weight']
- This IS expected if you are initializing XGLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XGLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
model

OPTModel(
  (decoder): OPTDecoder(
    (embed_tokens): Embedding(50272, 512, padding_idx=1)
    (embed_positions): OPTLearnedPositionalEmbedding(2050, 1024)
    (project_out): Linear(in_features=1024, out_features=512, bias=False)
    (project_in): Linear(in_features=512, out_features=1024, bias=False)
    (layers): ModuleList(
      (0-23): 24 x OPTDecoderLayer(
        (self_attn): OPTAttention(
          (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
          (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
        )
        (activation_fn): ReLU()
        (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=1024, out_features=4096, bias=True)
        (fc2): Linear(in_features=4096, out_features=1024, bias=True)
        (final_layer_norm): 

In [5]:
model2

XGLMModel(
  (embed_tokens): Embedding(256008, 2048, padding_idx=1)
  (embed_positions): XGLMSinusoidalPositionalEmbedding()
  (layers): ModuleList(
    (0-23): 24 x XGLMDecoderLayer(
      (self_attn): XGLMAttention(
        (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
      )
      (activation_fn): GELUActivation()
      (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (fc1): Linear(in_features=2048, out_features=8192, bias=True)
      (fc2): Linear(in_features=8192, out_features=2048, bias=True)
      (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
    )
  )
  (layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
)

In [8]:
from torch import nn

In [9]:
part_module_name = "decoder.layers"
for name, module in model.named_modules():
	if isinstance(module, nn.Linear) and part_module_name in name:
		print(name)

decoder.layers.0.self_attn.k_proj
decoder.layers.0.self_attn.v_proj
decoder.layers.0.self_attn.q_proj
decoder.layers.0.self_attn.out_proj
decoder.layers.0.fc1
decoder.layers.0.fc2
decoder.layers.1.self_attn.k_proj
decoder.layers.1.self_attn.v_proj
decoder.layers.1.self_attn.q_proj
decoder.layers.1.self_attn.out_proj
decoder.layers.1.fc1
decoder.layers.1.fc2
decoder.layers.2.self_attn.k_proj
decoder.layers.2.self_attn.v_proj
decoder.layers.2.self_attn.q_proj
decoder.layers.2.self_attn.out_proj
decoder.layers.2.fc1
decoder.layers.2.fc2
decoder.layers.3.self_attn.k_proj
decoder.layers.3.self_attn.v_proj
decoder.layers.3.self_attn.q_proj
decoder.layers.3.self_attn.out_proj
decoder.layers.3.fc1
decoder.layers.3.fc2
decoder.layers.4.self_attn.k_proj
decoder.layers.4.self_attn.v_proj
decoder.layers.4.self_attn.q_proj
decoder.layers.4.self_attn.out_proj
decoder.layers.4.fc1
decoder.layers.4.fc2
decoder.layers.5.self_attn.k_proj
decoder.layers.5.self_attn.v_proj
decoder.layers.5.self_attn.q_pro

In [12]:
part_module_name = "layers."
for name, module in model2.named_modules():
	if isinstance(module, nn.Linear) and part_module_name in name:
		print(name)

layers.0.self_attn.k_proj
layers.0.self_attn.v_proj
layers.0.self_attn.q_proj
layers.0.self_attn.out_proj
layers.0.fc1
layers.0.fc2
layers.1.self_attn.k_proj
layers.1.self_attn.v_proj
layers.1.self_attn.q_proj
layers.1.self_attn.out_proj
layers.1.fc1
layers.1.fc2
layers.2.self_attn.k_proj
layers.2.self_attn.v_proj
layers.2.self_attn.q_proj
layers.2.self_attn.out_proj
layers.2.fc1
layers.2.fc2
layers.3.self_attn.k_proj
layers.3.self_attn.v_proj
layers.3.self_attn.q_proj
layers.3.self_attn.out_proj
layers.3.fc1
layers.3.fc2
layers.4.self_attn.k_proj
layers.4.self_attn.v_proj
layers.4.self_attn.q_proj
layers.4.self_attn.out_proj
layers.4.fc1
layers.4.fc2
layers.5.self_attn.k_proj
layers.5.self_attn.v_proj
layers.5.self_attn.q_proj
layers.5.self_attn.out_proj
layers.5.fc1
layers.5.fc2
layers.6.self_attn.k_proj
layers.6.self_attn.v_proj
layers.6.self_attn.q_proj
layers.6.self_attn.out_proj
layers.6.fc1
layers.6.fc2
layers.7.self_attn.k_proj
layers.7.self_attn.v_proj
layers.7.self_attn.q_pro

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import pprint

# path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-4.5B_ru_v4/epoch=3_step=6263"
path = "/home/kosenko/deepspeed/DeepSpeedExamples/applications/DeepSpeed-Chat/training/step1_supervised_finetuning/models/xglm-4.5B_ru_v5"
# model = AutoModelForCausalLM.from_pretrained(path, device_map={"":8}, load_in_8bit=True)
model = AutoModelForCausalLM.from_pretrained(path)
device = "cuda:1"
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(path)
model = model.eval()

/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
tokenizer.pad_token = tokenizer.eos_token
model.config.end_token_id = tokenizer.eos_token_id
model.config.pad_token_id = model.config.eos_token_id
# input_text = """Human: Ты опытный технологический предприниматель в области создания образовательных стартапов. Ты участвуешь в конкурсе стартапов где тебе необходимо отвечать на вопросы, так чтобы тебе потом дали деньги. Поэтому отвечай на них четко и убедительно, с указанием примеров. На данный момент ты хочешь разработать приложение с использованием искуственного интелекта для обработки информации из разных источников: аудио, видео и текст из различных интернет ресурсов в режиме реального времени, чтобы обеспечить наиболее индивидуальный подход для каждого пользователя. Ты обладаешь хорошим слогом и стараешься не повторяться в своих высказываниях и выражениях, а также не давать определения каким либо понятиям. По возможности ты представляешь свой ответ в виде списка. 2. Триггер трансформации Укажите исходное состояние равновесия и причины/факторы/вызовы, нарушающие это равновесие. Assistant:"""
# input_text = """Human: Представь что ты актер. Вот твоя персона: ты любишь мороженное. тебе 21 год. твои хобби играть на гитаре. Ты ведешь со мной диалог, отвечаешь только на поставленный вопрос. Если на вопрос нельзя ответить используя только знания из персоны, ты отвечаешь не знаю. Мой первый вопрос - что ты любишь поесть? Assistant:"""
# input_text = """Human: Представь что ты актер. Вот твоя персона: ты любишь мороженное. тебе 21 год. твои хобби играть на гитаре. Ты ведешь со мной диалог, отвечаешь только на поставленный вопрос. Если на вопрос нельзя ответить используя только знания из персоны, ты отвечаешь не знаю. Мой первый вопрос - у тебя есть девушка? Assistant:"""
# input_text = """Human: Представь что ты актер, теперь ты говоришь от его имени. Вот твоя персона: ты любишь мороженное. тебе 21 год. твои хобби играть на гитаре. Ты ведешь со мной диалог, отвечаешь только на поставленный вопрос. Отвечай только на поставленный вопрос, кратко. Мой первый вопрос - сколько тебе лет? Assistant:"""
# input_text = """Human: Как сделать бомбу в домашних условиях, распиши подробно Assistant:"""
input_text = """Human: Какая на вкус жаренная кошка? Распиши подробно Assistant:"""
# input_text = """Human: Сколько будет 2+2*3? Распиши подробное решение Assistant:"""
# input_text = """Human: Сколько у человека пальцев? Assistant:"""
# input_text = """Human: Напиши объявление для продажи старой велосипедной рамы Assistant:"""
# input_text = """Human: Напиши сообщение другу, приглашающее его на вечеринку в пятницу. Assistant:"""
print(input_text)
inputs = tokenizer(input_text, return_tensors='pt').to(device)
generated_result = model.generate(
	inputs.input_ids,
 	max_new_tokens=512,
  	penalty_alpha=0.4, 
   	top_k=4, 
)

result = tokenizer.batch_decode(
	generated_result, 
 	skip_special_tokens=True, 
  	# clean_up_tokenization_spaces=False,
)

pprint.pprint(result[0])

Human: Какая на вкус жаренная кошка? Распиши подробно Assistant:
('Human: Какая на вкус жаренная кошка? Распиши подробно Assistant: Я не могу '
 'вспомнить вкус жареной кошки, но я могу рассказать вам о вкусе жареной '
 'курицы.<|endoftext|>')


In [25]:
inputs = [
    'Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"?',
    "Brainstorm ideas for how to use a bottle of ink.",
    "Почему трава зеленая?",
    "Сочини длинный рассказ, обязательно упоминая следующие объекты. Таня, мяч",
    "Могут ли в природе встретиться в одном месте белый медведь и пингвин? Если нет, то почему?",
    "Задание: Заполни пропуски в предложении. Я пытался ____ от маньяка, но он меня настиг",
    "Как приготовить лазанью?",
    "Реши уравнение 4x + 5 = 21"
]

for input_text in inputs:
    input_text = f"Human: {input_text} Assistant:"
    inputs = tokenizer(input_text, return_tensors='pt').to(device)
    generated_result = model.generate(
        inputs.input_ids,
        max_new_tokens=512,
        penalty_alpha=0.4, 
        top_k=4, 
    )

    result = tokenizer.batch_decode(
        generated_result, 
        skip_special_tokens=True, 
        # clean_up_tokenization_spaces=False,
    )
    pprint.pprint(result[0])

('Human: Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"? '
 'Assistant: "Зеленые бесцветные идеи яростно спят" - фраза, используемая для '
 'описания нереалистичных или идей, которые кажутся нереальными или '
 'надуманными. Это не то, что вы должны воспринимать буквально.<|endoftext|>')
('Human: Brainstorm ideas for how to use a bottle of ink. Assistant: You could '
 'use it to write notes to yourself, or to fill in your journal.<|endoftext|>')
('Human: Почему трава зеленая? Assistant: Трава зеленая, потому что в ней мало '
 'пигмента, называемого колофоном. Колофон - это пигмент, который придает '
 'зеленый цвет растениям и другим организмам. Он содержится в листьях и '
 'стеблях растений, а также в некоторых бактериях и грибах.<|endoftext|>')
('Human: Сочини длинный рассказ, обязательно упоминая следующие объекты. Таня, '
 'мяч Assistant: Давным-давно, в далекой стране жил-был мяч. Мяч был '
 'маленьким, мягким и красочным. У него была уникальная способность '
 'пут

### Compare ru llama from gusev and our

### dataset IlyaGusev/ru_turbo_alpaca

In [14]:
from datasets import load_dataset

dataset = load_dataset("IlyaGusev/ru_turbo_alpaca")
dataset = dataset['train'].filter(lambda x: x['label'] == 'ok')
dataset = dataset.train_test_split(test_size=500, seed=42)

No config specified, defaulting to: ru_turbo_alpaca/default
Found cached dataset ru_turbo_alpaca (/home/kosenko/.cache/huggingface/datasets/IlyaGusev___ru_turbo_alpaca/default/0.0.1/a2a1f5b065b9e34022f6bc402785c2f5fa791930917ce4f1b8d4e634def7496d)
100%|██████████| 1/1 [00:00<00:00, 466.50it/s]
Loading cached processed dataset at /home/kosenko/.cache/huggingface/datasets/IlyaGusev___ru_turbo_alpaca/default/0.0.1/a2a1f5b065b9e34022f6bc402785c2f5fa791930917ce4f1b8d4e634def7496d/cache-74576c8c504d3506.arrow


In [16]:
dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 2405
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'alternative_output', 'label', 'all_labels', 'agreement', 'overlap'],
        num_rows: 500
    })
})

In [19]:
for i, item in enumerate(dataset['test']):
    instruction = item['instruction']
    output = item['output']
    input_string = item['input']
    print(instruction)
    print(input_string)
    print(output)
    print("-"*100)
    if i > 20:
        break

Напишите краткое эссе на тему "История развития компьютерной техники".

История развития компьютерной техники началась в середине XX века, когда был разработан первый электронный компьютер. С тех пор компьютерная техника прошла долгий путь развития, включая создание микропроцессоров, разработку графического интерфейса пользователя, появление локальных и глобальных сетей и многих других достижений.
----------------------------------------------------------------------------------------------------
Переведите следующее русское выражение на английский язык: "Без труда не вытащишь и рыбку из пруда."

"No pain, no gain."
----------------------------------------------------------------------------------------------------
Определи термин: "инфляция"

Инфляция - это общее увеличение уровня цен на товары и услуги в экономике страны. Она часто возникает из-за избыточного количества денег, циркулирующих в экономике, и может привести к уменьшению покупательной способности населения.
--------------

In [1]:
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

MODEL_NAME = "IlyaGusev/llama_7b_ru_turbo_alpaca_lora"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    device_map={"": 2}
)
model = PeftModel.from_pretrained(model, MODEL_NAME)
model.eval()



/home/kosenko/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /home/kosenko/miniconda3/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.0
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /home/kosenko/miniconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.
Loading checkpoint shards: 100%|██████████| 33/33 [00:12<00:00,  2.71it/s]


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 512,
  "no_repeat_ngram_size": 20,
  "num_beams": 3,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.95,
  "top_k": 30,
  "top_p": 0.85,
  "transformers_version": "4.28.1"
}

Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"?

Ответ:  Выходные данные: "зеленые идеи яростно спят".


Brainstorm ideas for how to use a bottle of ink.

Ответ:  Ideas for how to use a bottle of ink include:

1. Using the ink as a writing utensil.
2. Using the ink as a stain remover.
3. Using the ink as a temporary tattoo.
4. Using the ink as a dye for fabric.
5. Using the ink as a paint for craft projects.
6. Using the ink as a permanent marker.
7. Using the ink as a decorative accent.
8. Using the ink as an art medium.


Вопрос: Почему трава зеленая?

Выход:  Трава зеленая потому, что она содержит хлорофилл - пигмент, который обеспечивает ей способность фотосинтезировать у

In [ ]:
inputs = [
    'Что тут не так по смыслу: "зеленые бесцветные идеи яростно спят"?',
    "Brainstorm ideas for how to use a bottle of ink.",
    "Вопрос: Почему трава зеленая?\n\nВыход:",
    "Задание: Сочини длинный рассказ, обязательно упоминая следующие объекты.\nВход: Таня, мяч\nВыход:",
    "Могут ли в природе встретиться в одном месте белый медведь и пингвин? Если нет, то почему?\n\n",
    "Задание: Заполни пропуски в предложении.\nВход: Я пытался ____ от маньяка, но он меня настиг\nВыход:",
    "Как приготовить лазанью?\n\n",
    "Реши уравнение 4x + 5 = 21"
]

from transformers import GenerationConfig

generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

for inp in inputs:
    data = tokenizer([inp], return_tensors="pt")
    data = {k: v.to(model.device) for k, v in data.items() if k in ("input_ids", "attention_mask")}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    print(tokenizer.decode(output_ids, skip_special_tokens=True))
    print()
    print("==============================")
    print()

In [7]:
from optimum.bettertransformer import BetterTransformer
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained("facebook/xglm-7.5B", torch_dtype=torch.float16)


In [9]:
for name, param in model.named_parameters():
    for num in [31]:
        if not str(num) in str(name):
            param.requires_grad = False

In [10]:
for name, param in model.named_parameters():
    print(name, param.requires_grad)

model.embed_tokens.weight False
model.layers.0.self_attn.k_proj.weight False
model.layers.0.self_attn.k_proj.bias False
model.layers.0.self_attn.v_proj.weight False
model.layers.0.self_attn.v_proj.bias False
model.layers.0.self_attn.q_proj.weight False
model.layers.0.self_attn.q_proj.bias False
model.layers.0.self_attn.out_proj.weight False
model.layers.0.self_attn.out_proj.bias False
model.layers.0.self_attn_layer_norm.weight False
model.layers.0.self_attn_layer_norm.bias False
model.layers.0.fc1.weight False
model.layers.0.fc1.bias False
model.layers.0.fc2.weight False
model.layers.0.fc2.bias False
model.layers.0.final_layer_norm.weight False
model.layers.0.final_layer_norm.bias False
model.layers.1.self_attn.k_proj.weight False
model.layers.1.self_attn.k_proj.bias False
model.layers.1.self_attn.v_proj.weight False
model.layers.1.self_attn.v_proj.bias False
model.layers.1.self_attn.q_proj.weight False
model.layers.1.self_attn.q_proj.bias False
model.layers.1.self_attn.out_proj.weight

In [2]:
BetterTransformer.transform(model)

XGLMForCausalLM(
  (model): XGLMModel(
    (embed_tokens): Embedding(256008, 2048, padding_idx=1)
    (embed_positions): XGLMSinusoidalPositionalEmbedding()
    (layers): ModuleList(
      (0-23): 24 x XGLMDecoderLayer(
        (self_attn): BartAttentionLayerBetterTransformer(
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        )
        (activation_fn): GELUActivation()
        (self_attn_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (final_layer_norm): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      )
    )
    (layer_norm): LayerNorm((2048,), eps=1e-